In [1]:
import carla
import random
from multiprocessing import Process
import queue
import time

### Utils functions

In [2]:
SENSOR_TICK = 3

def get_cam_blueprint(world, vehicle_id):
    cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    cam_bp.set_attribute("image_size_x",str(400 + vehicle_id))
    cam_bp.set_attribute("image_size_y",str(300))
    cam_bp.set_attribute("fov",str(100))
    cam_bp.set_attribute("sensor_tick",str(SENSOR_TICK))
    return cam_bp


def get_lidar_blueprint(world, vehicle_id):
    lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
    lidar_bp.set_attribute('sensor_tick', str(SENSOR_TICK))
    #lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('channels', str(vehicle_id))
    lidar_bp.set_attribute('points_per_second', '1120000')
    lidar_bp.set_attribute('upper_fov', '30')
    lidar_bp.set_attribute('range', '100')
    lidar_bp.set_attribute('rotation_frequency', '20')
    return lidar_bp

def geolocation_blueprint(world, vehicle_id):
    gnss_bp = world.get_blueprint_library().find('sensor.other.gnss')
    gnss_bp.set_attribute('sensor_tick', str(SENSOR_TICK))
    return gnss_bp


def retrieve_data(sensor_queue, timeout=5):
    while True:
        try:
            data = sensor_queue.get(True,timeout)
        except queue.Empty:
            return None
        return data

### Creating client

In [3]:
# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
# load town 04
client.load_world('Town04')

In [5]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# for blueprint in vehicle_blueprints:
#     print(blueprint.id)

In [6]:
# vehicle.mini.cooper_s
list(vehicle_blueprints)[-1]

### Add vehicles to world

In [12]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

vehicle_num = 10

# add vehicle_num number of vehicles at random spawn_points
for i in range(vehicle_num):
    world.try_spawn_actor(list(vehicle_blueprints)[-1], random.choice(spawn_points))

### Add sensors to cars

In [14]:
camera_init_trans = carla.Transform(carla.Location(z=1.7))
lidar_init_trans = carla.Transform(carla.Location(z=3.0))
spawned_vehicles = world.get_actors().filter('*vehicle*')


In [9]:
cam_queue = queue.Queue()
lidar_queue = queue.Queue()
cam_list = []
lidar_list = []
for vehicle in spawned_vehicles:
    print(vehicle)

    # create camera and attach to vehicle
    camera_bp = get_cam_blueprint(world, vehicle.id)
    camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
    #camera.listen(lambda image: image.save_to_disk(f'sensors/vehicle-id-{vehicle.id}-%06d.png' % image.frame))
    camera.listen(cam_queue.put)
    cam_list.append(camera)

    # create lidar sensor and attach to vehicle
    lidar_bp = get_lidar_blueprint(world, vehicle.id)
    lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)
    lidar.listen(lidar_queue.put)
    lidar_list.append(lidar)

    # add vehicles to Trafic manager
    vehicle.set_autopilot(True)
    
    ## TODO: ADD GNSS for gps and location
    ## TODO: ADD IMU for speed

### Local part where the data is processed

At this point, data is saved at storage

In the future, this queues will be MQ queues to communicate with OMNET+

In [10]:
time_sim = 0
while True:

    # sleep before getting data from queues
    time.sleep(1)
    
    # get cam data
    data = retrieve_data(cam_queue)
    # Skip if any sensor data is not available
    if None in data:
        ...
    else:
        rgb_img = data
        # print(rgb_img)
        rgb_img.save_to_disk(f'sensors/camera/vehicle-{rgb_img.width - 400}-camera-frame-%06d.png' % rgb_img.frame)
    
    # get lidar data
    lidar_data = retrieve_data(lidar_queue)
    if None in data:
        ...
    else:
        #print("lidar data:", lidar_data)
        lidar_data.save_to_disk(f'sensors/lidar/vehicle-{lidar_data.channels}-frame-%06d.ply' % rgb_img.frame)

In [15]:
# for vehicle in spawned_vehicles:
#     print(vehicle)

#     # add vehicles to Trafic manager
#     vehicle.set_autopilot(True)


Actor(id=310, type=vehicle.mini.cooper_s)
Actor(id=309, type=vehicle.mini.cooper_s)
Actor(id=308, type=vehicle.mini.cooper_s)
Actor(id=307, type=vehicle.mini.cooper_s)
Actor(id=306, type=vehicle.mini.cooper_s)
Actor(id=305, type=vehicle.mini.cooper_s)
Actor(id=304, type=vehicle.mini.cooper_s)
Actor(id=303, type=vehicle.mini.cooper_s)
Actor(id=302, type=vehicle.mini.cooper_s)
Actor(id=301, type=vehicle.mini.cooper_s)
Actor(id=300, type=vehicle.mini.cooper_s)
Actor(id=299, type=vehicle.mini.cooper_s)
Actor(id=298, type=vehicle.mini.cooper_s)
Actor(id=297, type=vehicle.mini.cooper_s)
Actor(id=296, type=vehicle.mini.cooper_s)
Actor(id=295, type=vehicle.mini.cooper_s)
Actor(id=294, type=vehicle.mini.cooper_s)
Actor(id=293, type=vehicle.mini.cooper_s)
Actor(id=292, type=vehicle.mini.cooper_s)
Actor(id=291, type=vehicle.mini.cooper_s)
Actor(id=290, type=vehicle.mini.cooper_s)
Actor(id=289, type=vehicle.mini.cooper_s)
Actor(id=288, type=vehicle.mini.cooper_s)
Actor(id=287, type=vehicle.mini.co